# Imports.

In [1]:
import numpy as np
import pandas as pd
from pymoo.algorithms.moo.dnsga2 import DNSGA2
from pymoo.core.callback import CallbackCollection, Callback
from pymoo.optimize import minimize
from pymoo.problems.dyn import TimeSimulation
from pymoo.problems.static import StaticProblem
from pymoo.problems.dynamic.df import DF1,DF2,DF3,DF4,DF5,DF6,DF7,DF8,DF9,DF10,DF11,DF12,DF13,DF14

# Automate Building the Datasets (using default values at the moment).

In [2]:
# DF 10 --14  3 objectives
# 1--> 9 2 obj
class ObjectiveSpaceAnimation(Callback):
            def __init__(self):
                self.X = []
                self.X_star = []
                self.F = []
                self.ttau = []
                #self.hist_F = []

            def _update(self, algorithm):
                self.X.append(algorithm.pop.get("X"))
                self.X_star.append(algorithm.opt.get("X"))
                
                #feas = np.where(algorithm.opt.get("feasible"))[0]
                #self.X_star.append(algorithm.opt.get("F")[feas])    
                
                self.F.append(algorithm.pop.get("F"))
                self.ttau.append(algorithm.problem.tau)

            def pareto_set(self):
                return np.array(self.X, dtype= object)

            def pareto_optimal_set(self):
                return np.array(self.X_star, dtype= object)

            def pareto_front(self):
                return np.array(self.F, dtype= object)

# Note all our POSs are not in equal shape (aka 500 if nbr_pop=500)
# We need to make sure that they are with the same size
# therefore we need to store some POSs to add them later on
# we choose 0.2 random POSs and POSs correspond of the minimum POFs 
def POSs_to_be_added(problem , POS, percentage = 0.2):
    out = {} 
    problem._evaluate(POS, out)
    # Select the minimums in all the columns (with their indexes)
    # out["F"] is the corresponding current POF of our current POS
    #minimum_POF_values = np.min(out["F"], axis=0) 
    minimum_POF_values_index = np.argmin(out["F"], axis=0) 
    minimum_POSs = POS[minimum_POF_values_index]

    # Select a random number of elements from 'out["F"]'
    # 20% of elements

    num_elements = int(out["F"].shape[0] * percentage)

    rand_indexes = np.random.choice(out["F"].shape[0], num_elements, replace=False)
    random_POSs = POS[rand_indexes]
    #rand_selected_elements = out["F"][rand_indexes, :]

    return minimum_POSs, random_POSs#, minimum_POF_values

def create_datasets(nbr_generations = 100, nbr_pop=100, percentage=0.8):
    DFs = [DF1, DF2, DF3, DF4, DF5, DF6, DF7, DF8, DF9, DF10, DF11, DF12, DF13, DF14] 
    PSs = []
    POSs = []
    reference_PF = []
    n_evals = [] 
    minimum_values = []
    minimum_values_index = []
    selected_elements = []
    selected_indexes = []
    
    for i in range(len(DFs)):
        problem = DFs[i]()
        reference_PF.append( problem.pareto_front(use_cache=False, flatten=True) )
        
        
        print('################################')
        print('############# DF',i+1,'#############')
        print('################################')
        
        
        
        algorithm = DNSGA2(pop_size=nbr_pop)
        n_evals.append(algorithm.evaluator.n_eval)
        
        simulation = TimeSimulation()
        
        objective_space_animation = ObjectiveSpaceAnimation()
        
        res = minimize(problem,
                       algorithm,
                       termination=('n_gen', nbr_generations),
                       callback=CallbackCollection(objective_space_animation, simulation),
                       seed=1,
                       verbose=True)
        
        #print(objective_space_animation.ttau)
        # Used those function for the numpy arrays  
        current_PS = objective_space_animation.pareto_set() 
        PSs.append( current_PS )
        current_POS = objective_space_animation.pareto_optimal_set()
        POSs.append( current_POS )
        
        min_POSs = []
        rand_POSs = []
        for j in range(len(current_POS)):
            #print(current_POS[j].shape, j, 'DF',i+1)
            minimum_POSs, random_POSs = POSs_to_be_added(problem, current_POS[j], percentage)
            min_POSs.append(minimum_POSs)
            rand_POSs.append(random_POSs)
        
        
    return PSs, POSs, reference_PF, n_evals, min_POSs, rand_POSs

In [3]:
%%time
# We Need to increase the nbr of generations.
PSs, POSs, reference_PF, n_evals, min_POSs, rand_POSs = create_datasets(nbr_generations = 500, nbr_pop=500)

################################
############# DF 1 #############
################################
n_gen  |  n_eval  | n_nds  |      igd      |       gd      |       hv     
     1 |      500 |     10 |  0.8510136345 |  1.7805685314 |  0.000000E+00
     2 |     1550 |     12 |  0.7624675311 |  0.8281863052 |  0.000000E+00
     3 |     2600 |     10 |  0.4834524611 |  0.6398879543 |  0.0163813502
     4 |     3650 |     12 |  0.2966598819 |  0.4806878733 |  0.1112187843
     5 |     4700 |     16 |  0.2688674120 |  0.2676832692 |  0.1507442633
     6 |     5750 |     19 |  0.1820597229 |  0.2605573886 |  0.2146224041
     7 |     6800 |     20 |  0.1564478442 |  0.2624130596 |  0.2485738448
     8 |     7850 |     25 |  0.0830942042 |  0.1951900370 |  0.3313165524
     9 |     8900 |     29 |  0.0721266910 |  0.1378394952 |  0.3442216296
    10 |     9950 |     44 |  0.0592422274 |  0.1143422736 |  0.3608867155
    11 |    11000 |     48 |  0.0493611640 |  0.0691821576 |  0.3757200248
 

   106 |   113250 |     94 |  0.0959486798 |  0.1019128295 |  0.3010090210
   107 |   114300 |    108 |  0.0913260812 |  0.0988180570 |  0.3077380661
   108 |   115350 |    113 |  0.0856105899 |  0.0946570027 |  0.3143014984
   109 |   116400 |    116 |  0.0807637688 |  0.0877006406 |  0.3212088330
   110 |   117450 |    115 |  0.0756266163 |  0.0843246433 |  0.3271009518
   111 |   118500 |    119 |  0.0714653934 |  0.0765334002 |  0.3332253271
   112 |   119550 |    145 |  0.0695707656 |  0.0772549490 |  0.3372197690
   113 |   120600 |    152 |  0.0670622839 |  0.0716226686 |  0.3409287919
   114 |   121650 |    169 |  0.0650938796 |  0.0686885194 |  0.3442325716
   115 |   122700 |    206 |  0.0636356878 |  0.0641104105 |  0.3470525336
   116 |   123750 |    237 |  0.0628369774 |  0.0652246727 |  0.3488523001
   117 |   124800 |    265 |  0.0619451676 |  0.0663132245 |  0.3503709433
   118 |   125850 |    311 |  0.0613613450 |  0.0668999528 |  0.3515157366
   119 |   126900 |    32

   216 |   231250 |    500 |  0.0755595873 |  0.0798771652 |  0.3317842676
   217 |   232300 |    500 |  0.0755465536 |  0.0798778794 |  0.3318083080
   218 |   233350 |    500 |  0.0755250501 |  0.0804191953 |  0.3318552221
   219 |   234400 |    500 |  0.0755077399 |  0.0799446970 |  0.3318696722
   220 |   235950 |    431 |  0.0753208635 |  0.0809784827 |  0.3317440170
   221 |   237000 |    500 |  0.0752547169 |  0.0803288297 |  0.3320915469
   222 |   238050 |    500 |  0.0752389208 |  0.0795299500 |  0.3322506295
   223 |   239100 |    500 |  0.0752106360 |  0.0793624856 |  0.3323049921
   224 |   240150 |    500 |  0.0751907171 |  0.0789320887 |  0.3323418763
   225 |   241200 |    500 |  0.0751786261 |  0.0794040104 |  0.3323802597
   226 |   242250 |    500 |  0.0751525260 |  0.0779397552 |  0.3324146852
   227 |   243300 |    500 |  0.0751390180 |  0.0788695138 |  0.3324326378
   228 |   244350 |    500 |  0.0751346947 |  0.0791095621 |  0.3324948145
   229 |   245400 |    50

   326 |   349750 |     92 |  0.0910724450 |  0.1025125678 |  0.3054515527
   327 |   350800 |     92 |  0.0852177077 |  0.0947051286 |  0.3142551126
   328 |   351850 |    101 |  0.0796504522 |  0.0862692562 |  0.3225941688
   329 |   352900 |    112 |  0.0746533304 |  0.0809095292 |  0.3296517816
   330 |   353950 |    112 |  0.0708557568 |  0.0750808817 |  0.3356467406
   331 |   355000 |    123 |  0.0667416624 |  0.0723339283 |  0.3417793769
   332 |   356050 |    147 |  0.0630387140 |  0.0698832377 |  0.3465121364
   333 |   357100 |    161 |  0.0601914610 |  0.0673911466 |  0.3507362969
   334 |   358150 |    181 |  0.0584920836 |  0.0653135764 |  0.3542458636
   335 |   359200 |    211 |  0.0568949151 |  0.0636516903 |  0.3572416741
   336 |   360250 |    236 |  0.0552653015 |  0.0592748019 |  0.3596992398
   337 |   361300 |    274 |  0.0545680546 |  0.0587490254 |  0.3612518926
   338 |   362350 |    302 |  0.0538973599 |  0.0593206401 |  0.3626425611
   339 |   363400 |    32

   437 |   468800 |    284 |  0.0114843119 |  0.0132428047 |  0.4582791578
   438 |   469850 |    335 |  0.0118547583 |  0.0135012236 |  0.4597047181
   439 |   470900 |    372 |  0.0122391756 |  0.0139949960 |  0.4607872570
   440 |   472450 |     65 |  0.0816138706 |  0.0933931522 |  0.3311378569
   441 |   473500 |     56 |  0.0739961384 |  0.0769599136 |  0.3408618723
   442 |   474550 |     61 |  0.0618787203 |  0.0683319190 |  0.3548230647
   443 |   475600 |     67 |  0.0473615255 |  0.0582248205 |  0.3710919063
   444 |   476650 |     64 |  0.0362972539 |  0.0441915801 |  0.3895209960
   445 |   477700 |     73 |  0.0273021534 |  0.0327854263 |  0.4032741899
   446 |   478750 |     76 |  0.0201756765 |  0.0260173098 |  0.4181503389
   447 |   479800 |     83 |  0.0163183314 |  0.0187862926 |  0.4297658562
   448 |   480850 |     83 |  0.0135242882 |  0.0157624747 |  0.4401467968
   449 |   481900 |     98 |  0.0148066262 |  0.0158852598 |  0.4494175438
   450 |   482950 |    10

...

   433 |   464600 |    500 |  0.1025347326 |  0.1523238830
   434 |   465650 |    500 |  0.1035253397 |  0.1547769980
   435 |   466700 |    500 |  0.1044597992 |  0.1525074228
   436 |   467750 |    500 |  0.1071384682 |  0.1515383882
   437 |   468800 |    500 |  0.1079906852 |  0.1514947834
   438 |   469850 |    500 |  0.1107758649 |  0.1542168511
   439 |   470900 |    500 |  0.1121513045 |  0.1541884149
   440 |   472450 |    195 |  0.1209591366 |  0.2883408857
   441 |   473500 |    236 |  0.1136440762 |  0.2666340747
   442 |   474550 |    292 |  0.1126685797 |  0.2376958888
   443 |   475600 |    361 |  0.1093223947 |  0.2176521163
   444 |   476650 |    401 |  0.1035591332 |  0.1983464902
   445 |   477700 |    418 |  0.1023490606 |  0.1779420129
   446 |   478750 |    434 |  0.1022963950 |  0.1691868432
   447 |   479800 |    468 |  0.1049036972 |  0.1670914718
   448 |   480850 |    500 |  0.1123563753 |  0.1655506078
   449 |   481900 |    500 |  0.1133018662 |  0.15604770

    69 |    73400 |    413 |  0.0472935787 |  0.2252481800
    70 |    74450 |    410 |  0.0452678451 |  0.2156761344
    71 |    75500 |    467 |  0.0418197686 |  0.2209098845
    72 |    76550 |    485 |  0.0390385514 |  0.2135594355
    73 |    77600 |    500 |  0.0393206156 |  0.2026488624
    74 |    78650 |    500 |  0.0372174626 |  0.2028015330
    75 |    79700 |    500 |  0.0328053777 |  0.2082469000
    76 |    80750 |    500 |  0.0322583246 |  0.2151783910
    77 |    81800 |    500 |  0.0351703784 |  0.2091149646
    78 |    82850 |    500 |  0.0346452761 |  0.2153324429
    79 |    83900 |    500 |  0.0330901072 |  0.2054069005
    80 |    85450 |    237 |  0.1178645851 |  0.3739478928
    81 |    86500 |    292 |  0.1087393506 |  0.3758644232
    82 |    87550 |    337 |  0.1028486330 |  0.3729364072
    83 |    88600 |    359 |  0.0912548763 |  0.3766303423
    84 |    89650 |    433 |  0.0836716887 |  0.3884050234
    85 |    90700 |    439 |  0.0745545766 |  0.37544457

   208 |   222850 |    500 |  0.1940701959 |  3.018753E+01
   209 |   223900 |    500 |  0.3756980533 |  3.098169E+01
   210 |   224950 |    500 |  0.3628341408 |  3.091092E+01
   211 |   226000 |    500 |  0.3628341408 |  3.066980E+01
   212 |   227050 |    500 |  0.2648785947 |  3.031799E+01
   213 |   228100 |    500 |  0.2648785947 |  3.064272E+01
   214 |   229150 |    500 |  0.2645925065 |  3.051881E+01
   215 |   230200 |    500 |  0.2629538847 |  3.052136E+01
   216 |   231250 |    500 |  0.1334078266 |  3.054944E+01
   217 |   232300 |    500 |  0.1346928360 |  2.921993E+01
   218 |   233350 |    500 |  0.1307883461 |  2.948386E+01
   219 |   234400 |    500 |  0.1339648886 |  2.891163E+01
   220 |   235950 |    500 |  0.1272187800 |  2.804879E+01
   221 |   237000 |    500 |  0.1193123416 |  2.827870E+01
   222 |   238050 |    500 |  0.0946894197 |  2.732566E+01
   223 |   239100 |    500 |  0.0921141879 |  2.598106E+01
   224 |   240150 |    500 |  0.0838136719 |  2.507867E+

   347 |   372300 |    500 |  0.1118979539 |  1.161517E+01
   348 |   373350 |    500 |  0.1115901716 |  1.186095E+01
   349 |   374400 |    500 |  0.1089194865 |  1.194886E+01
   350 |   375450 |    500 |  0.1050089744 |  1.188988E+01
   351 |   376500 |    500 |  0.1030935792 |  1.160886E+01
   352 |   377550 |    500 |  0.1034526418 |  1.178687E+01
   353 |   378600 |    500 |  0.1026688145 |  1.160223E+01
   354 |   379650 |    500 |  0.0901962087 |  1.131827E+01
   355 |   380700 |    500 |  0.0851156879 |  1.165780E+01
   356 |   381750 |    500 |  0.0820985124 |  1.123901E+01
   357 |   382800 |    500 |  0.0722324587 |  1.093709E+01
   358 |   383850 |    500 |  0.0666018380 |  1.116472E+01
   359 |   384900 |    500 |  0.0650052543 |  1.077456E+01
   360 |   386450 |    414 |  0.2052766159 |  9.0459831686
   361 |   387500 |    500 |  0.1855817742 |  8.9138879312
   362 |   388550 |    500 |  0.1805375402 |  8.8223591876
   363 |   389600 |    500 |  0.1717153544 |  8.91514269

   487 |   522800 |    500 |  0.1358388262 |  1.427226E+01
   488 |   523850 |    500 |  0.1126241817 |  1.403102E+01
   489 |   524900 |    500 |  0.1040581652 |  1.400984E+01
   490 |   525950 |    500 |  0.1076331363 |  1.424373E+01
   491 |   527000 |    500 |  0.0981409932 |  1.380974E+01
   492 |   528050 |    500 |  0.0995212067 |  1.370838E+01
   493 |   529100 |    500 |  0.0868818453 |  1.324460E+01
   494 |   530150 |    500 |  0.0825975193 |  1.313940E+01
   495 |   531200 |    500 |  0.0944814432 |  1.306265E+01
   496 |   532250 |    500 |  0.0915038608 |  1.329497E+01
   497 |   533300 |    500 |  0.0792259116 |  1.294669E+01
   498 |   534350 |    500 |  0.0722057444 |  1.253578E+01
   499 |   535400 |    500 |  0.0737392915 |  1.274548E+01
   500 |   536950 |    429 |  0.1614771833 |  1.663178E+01
Wall time: 13min 53s


In [4]:
reference_PF = np.array(reference_PF, dtype=object)
n_evals = np.array(n_evals, dtype=object)

In [5]:
path1 = 'my_dataset/reference_PF/'
np.save(file=path1+'reference_PF',arr=reference_PF)
#reference_PF, n_evals,

In [6]:
path2 = 'my_dataset/n_evals/'
np.save(file=path2+'n_evals', arr=n_evals)

In [7]:
min_POSs = np.array(min_POSs)
rand_POSs = np.array(rand_POSs, dtype=object)

In [8]:
np.array(min_POSs).shape, np.array(rand_POSs, dtype=object).shape

((500, 3, 10), (500,))

In [9]:
len(rand_POSs[0]), rand_POSs[0]

(11,
 array([[ 0.22604417,  0.00111483,  0.37143314,  0.90412508, -0.21769648,
         -0.39940314,  0.10750363,  0.70395252,  0.58993485,  0.36951341],
        [ 0.87999829,  0.90384196,  0.32543962, -0.45958348, -0.4952666 ,
          0.70979589,  0.05542929,  0.60432217,  0.14497703,  0.46628505],
        [ 0.80300004,  0.99614144, -0.77635152,  0.06209471,  0.62705316,
         -0.24741163, -0.10845095,  0.50180512, -0.80330724, -0.27894739],
        [ 0.00956689,  0.25195142,  0.01135293,  0.0808333 , -0.25741515,
         -0.41457725, -0.59526531,  0.60056379, -0.36815419, -0.01506403],
        [ 0.03762695,  0.58303392,  0.34511362, -0.07903282,  0.48223371,
          0.00204993,  0.40470524, -0.53871819,  0.48922482,  0.0032744 ],
        [ 0.75127164,  0.88984136, -0.08917961,  0.19881218, -0.17309345,
         -0.39148967,  0.05094114, -0.61806538, -0.57179655,  0.02157484],
        [ 0.21077094,  0.50282438, -0.79009266, -0.22225319,  0.07597544,
         -0.32159163,  0.11

In [10]:
len(min_POSs[0]), min_POSs[0][0].shape

(3, (10,))

In [11]:
len(rand_POSs[0]), rand_POSs[0]

(11,
 array([[ 0.22604417,  0.00111483,  0.37143314,  0.90412508, -0.21769648,
         -0.39940314,  0.10750363,  0.70395252,  0.58993485,  0.36951341],
        [ 0.87999829,  0.90384196,  0.32543962, -0.45958348, -0.4952666 ,
          0.70979589,  0.05542929,  0.60432217,  0.14497703,  0.46628505],
        [ 0.80300004,  0.99614144, -0.77635152,  0.06209471,  0.62705316,
         -0.24741163, -0.10845095,  0.50180512, -0.80330724, -0.27894739],
        [ 0.00956689,  0.25195142,  0.01135293,  0.0808333 , -0.25741515,
         -0.41457725, -0.59526531,  0.60056379, -0.36815419, -0.01506403],
        [ 0.03762695,  0.58303392,  0.34511362, -0.07903282,  0.48223371,
          0.00204993,  0.40470524, -0.53871819,  0.48922482,  0.0032744 ],
        [ 0.75127164,  0.88984136, -0.08917961,  0.19881218, -0.17309345,
         -0.39148967,  0.05094114, -0.61806538, -0.57179655,  0.02157484],
        [ 0.21077094,  0.50282438, -0.79009266, -0.22225319,  0.07597544,
         -0.32159163,  0.11

In [12]:
np.array(PSs).shape, np.array(POSs).shape

((14, 500, 500, 10), (14, 500))

In [13]:
min_POSs[0].shape, rand_POSs[0].shape, len(min_POSs[0])

((3, 10), (11, 10), 3)

## Make Every Generation Compatible by Using POS_Padding Technique Discussed or a Simple Zero-Padding Technique(not advised).

In [43]:
def pos_paddings(POS, min_POSs, rand_POSs, nbr_pop=100, nbr_var=10):
    new_pos = POS.copy()
    current_df= []
    #counter = 0
    for i in range(new_pos.shape[0]):
        if(new_pos[i].shape[0] != nbr_pop):
            my_list = []
            j = 0
            # print(len(my_list))
            #print(( nbr_pop - new_pos[i].shape[0] ))
            while( len(my_list) != ( nbr_pop - new_pos[i].shape[0] ) ):
                #print('inside while loop')
                #print(j)
                if(j==0):
                    # can't append min_POSs all directly so i am doing it explicitly
                    for k in range( len(min_POSs) ):
                        #print(k, len(min_POSs))
                        my_list.append(min_POSs[k])
                        
                # same can't append rand_POSs all directly so i am doing it explicitly        
                for k in range(len(rand_POSs)):
                    #print(rand_POSs.shape)
                    my_list.append(rand_POSs[k])
                
                # implemented this to make sure that our resulted list(aka new_pos1) is always == pop_size 
                if( len(my_list) > ( nbr_pop - new_pos[i].shape[0] ) ):
                    nbr_we_dont_need = len(my_list) - ( nbr_pop - new_pos[i].shape[0] )
                    for _ in range(nbr_we_dont_need):
                        #print('in pop iteration n:',m)
                        my_list.pop()
                
                # improvements to do later on 
                # make some random value then if value > treshold append min_POSs else append rand_POSs
                # j==0 is certainly a lazy method but it works
                j = j + 1
                   
            my_list = np.array(my_list)
            #print(my_list.shape)
            #print(new_pos[i].shape, my_list.shape)
            new_pos1 = np.concatenate( (new_pos[i], my_list), axis=0) 
            #counter = counter + 1 
            #print(new_pos1.shape, counter)
            current_df.append(pd.DataFrame(new_pos1, index=None))
            #print(current_df[0].shape)
        else:
            current_df.append( pd.DataFrame(new_pos[i], index=None) )
            #counter = counter + 1

    df1 = pd.concat(current_df, ignore_index=True)
    return df1

In [44]:
%%time
df1 = pos_paddings(POSs[0], min_POSs[0], rand_POSs[0], nbr_pop=500)
df1

Wall time: 501 ms


,0,1,2,3,4,5,6,7,8,9
0,0.556240,0.136455,0.059918,0.121343,0.044552,0.107494,0.225709,0.712989,0.559717,0.012556
1,0.003113,0.774897,0.896417,0.239316,0.120767,0.220284,0.302097,0.883029,0.543166,0.286712
2,0.005797,0.312681,0.055698,0.293862,0.356016,0.953677,0.476951,0.278238,0.503005,0.643358
3,0.001033,0.476089,0.657040,0.843323,0.183739,0.856159,0.364365,0.892274,0.027712,0.085390
4,0.032909,0.113411,0.477461,0.024027,0.626209,0.097583,0.250962,0.426493,0.115827,0.270552
...,...,...,...,...,...,...,...,...,...,...
249995,0.803000,0.996141,-0.776352,0.062095,0.627053,-0.247412,-0.108451,0.501805,-0.803307,-0.278947
249996,0.009567,0.251951,0.011353,0.080833,-0.257415,-0.414577,-0.595265,0.600564,-0.368154,-0.015064
249997,0.037627,0.583034,0.345114,-0.079033,0.482234,0.002050,0.404705,-0.538718,0.489225,0.003274
249998,0.751272,0.889841,-0.089180,0.198812,-0.173093,-0.391490,0.050941,-0.618065,-0.571797,0.021575


In [45]:
# the old technique.
def zero_paddings(POS, nbr_pop=100, nbr_var=10):
    new_pos = POS.copy()
    current_df= []
    counter = 0
    for i in range(new_pos.shape[0]):
        if(new_pos[i].shape[0] != nbr_pop):
            my_list = []
            for j in range(nbr_pop - new_pos[i].shape[0]):
                new_list = []
                for _ in range(nbr_var):
                    new_list.append(0)
                new_list = np.array(new_list)
                my_list.append(new_list)

            my_list = np.array(my_list)
            #print(my_list.shape, new_pos[i].shape)
            new_pos1 = np.concatenate( (new_pos[i], my_list), axis=0) 
            counter = counter + 1 
            #print(new_pos1.shape, counter)
            current_df.append(pd.DataFrame(new_pos1, index=None))
            #print(current_df[0].shape)
        else:
            current_df.append( pd.DataFrame(new_pos[i], index=None) )
            counter = counter + 1

    df1 = pd.concat(current_df, ignore_index=True)
    return df1

In [28]:
# run an example
# generation 1: from 0 to 99 
# generation 2: from 100 to 199 
# generation 3: from 200 to 299 
# ...etc
df1 = zero_paddings(POSs[0], nbr_pop=500)
df1

,0,1,2,3,4,5,6,7,8,9
0,0.556240,0.136455,0.059918,0.121343,0.044552,0.107494,0.225709,0.712989,0.559717,0.012556
1,0.003113,0.774897,0.896417,0.239316,0.120767,0.220284,0.302097,0.883029,0.543166,0.286712
2,0.005797,0.312681,0.055698,0.293862,0.356016,0.953677,0.476951,0.278238,0.503005,0.643358
3,0.001033,0.476089,0.657040,0.843323,0.183739,0.856159,0.364365,0.892274,0.027712,0.085390
4,0.032909,0.113411,0.477461,0.024027,0.626209,0.097583,0.250962,0.426493,0.115827,0.270552
...,...,...,...,...,...,...,...,...,...,...
249995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
249996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
249997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
249998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
# Using pos_paddings and not zero paddings.
def preprocess_pos(POSs, nbr_pop=100): 
    pos_dfs = []
    for i in range(len(POSs)):
        #zero_padded_df = zero_paddings(POSs[i], nbr_pop=nbr_pop)
        pos_padded_df = pos_paddings(POSs[0], min_POSs[i], rand_POSs[i], nbr_pop=500)
        pos_dfs.append(pos_padded_df)#zero_padded_df)
        print('###############')
        print('POS nbr',i+1)
        print('###############')
    return pos_dfs

# Convert POS data to a DataFrame then to csv File.

In [35]:
# The laziest way possible.
def save_data(ps_dfs=[], pos_dfs=[], isPOS=False):
    n = 0
    if(isPOS == True):
        n = len(pos_dfs)
    else:
        n = len(ps_dfs)
    
    for i in range(n):
        if(isPOS == True):
            path = 'my_dataset/POS/pos_DF_' + str(i+1) + '.csv'
            pos_dfs[i].to_csv(path, index=False)
            print('###############')
            print('DF'+ str(i+1))
            print('###############')
        
        else:
            path = 'my_dataset/PS/ps_DF_' + str(i+1) + '.csv'
            ps_dfs[i].to_csv(path, index=False)
            print('###############')
            print('DF'+ str(i+1))
            print('###############')

# Use Pos_Paddings Then Save It Using Save_Data Function.

In [48]:
%%time
my_dfs = preprocess_pos(POSs, nbr_pop=500)
print(np.array(my_dfs).shape)
save_data( pos_dfs = my_dfs, isPOS=True)

###############
POS nbr 1
###############
###############
POS nbr 2
###############
###############
POS nbr 3
###############
###############
POS nbr 4
###############
###############
POS nbr 5
###############
###############
POS nbr 6
###############
###############
POS nbr 7
###############
###############
POS nbr 8
###############
###############
POS nbr 9
###############
###############
POS nbr 10
###############
###############
POS nbr 11
###############
###############
POS nbr 12
###############
###############
POS nbr 13
###############
###############
POS nbr 14
###############
(14, 250000, 10)
DF1
DF2
DF3
DF4
DF5
DF6
DF7
DF8
DF9
DF10
DF11
DF12
DF13
DF14
Wall time: 1min 9s


# Convert PS data to a DataFrame then to csv File.

In [39]:
def ps_to_data_frame(PSs):
    pss = PSs.copy()
    my_list = []
    my_df = []
    for i in range(pss.shape[0]):
        my_list.append( pd.DataFrame(pss[i], index=None) )
    my_df = pd.concat(my_list, ignore_index=True)
    return my_df

def create_multiple_dfs_of_ps(PSs):
    my_pss = PSs.copy()
    my_DF = []
    for i in range(len(my_pss)):
        my_DF.append(ps_to_data_frame(my_pss[i]))
    return my_DF

In [40]:
np.array(PSs)[0][0].shape

(500, 10)

In [41]:
# running an example
my_DF = ps_to_data_frame(PSs[0])
my_DF

,0,1,2,3,4,5,6,7,8,9
0,0.55624,0.136455,0.059918,0.121343,0.044552,0.107494,0.225709,0.712989,0.559717,0.012556
1,0.003113,0.774897,0.896417,0.239316,0.120767,0.220284,0.302097,0.883029,0.543166,0.286712
2,0.005797,0.312681,0.055698,0.293862,0.356016,0.953677,0.476951,0.278238,0.503005,0.643358
3,0.001033,0.476089,0.65704,0.843323,0.183739,0.856159,0.364365,0.892274,0.027712,0.08539
4,0.032909,0.113411,0.477461,0.024027,0.626209,0.097583,0.250962,0.426493,0.115827,0.270552
...,...,...,...,...,...,...,...,...,...,...
249995,0.040694,0.569021,0.566724,0.571514,0.581549,0.515195,0.633984,0.551611,0.561803,0.572527
249996,0.953431,0.565195,0.620878,0.572099,0.579931,0.542475,0.560027,0.557621,0.57013,0.591908
249997,0.897502,0.584771,0.589156,0.601679,0.60469,0.547717,0.628729,0.553797,0.553583,0.567295
249998,0.3605,0.566437,0.554549,0.524599,0.582387,0.583276,0.628914,0.557856,0.599714,0.548135


In [42]:
%%time
my_DF = create_multiple_dfs_of_ps(PSs)
save_data(ps_dfs = my_DF, isPOS=False)

DF1
DF2
DF3
DF4
DF5
DF6
DF7
DF8
DF9
DF10
DF11
DF12
DF13
DF14
Wall time: 1min 6s


# DONE!,
### Created 2 (POSs and PSs) Datasets of 14 csv files (14 benchmarks aka DFs)
### each file with 500 generation of 500 populations
### of samples of 10 Variables (Default Values at the moment).